In [ ]:
# certain pair/sample set info was accidentally removed
# this notebook reverted those changes and removed samples we want to remove due to various issues
# see https://app.asana.com/0/478455687490660/1201271559247444/f
import dalmatian as dm

sandbox2ws = 'broad-firecloud-ccle/DepMap_WES_CN_hg38-sandbox2'
sandbox2wm = dm.WorkspaceManager(sandbox2ws).disable_hound()

In [ ]:
sandbox1ws = 'broad-firecloud-ccle/DepMap_WES_CN_hg38'
sandbox1wm = dm.WorkspaceManager(sandbox1ws).disable_hound()

In [ ]:
sb2pairs = sandbox2wm.get_pairs()
sb1pairs = sandbox1wm.get_pairs()

In [ ]:
pairstoadd = sb2pairs[~sb2pairs.index.isin(sb1pairs.index)]
pairstoadd

In [ ]:
sandbox1wm.upload_entities('pair', pairstoadd)

In [ ]:
sandbox1wm.get_sample_sets()

In [ ]:
for s in sandbox1wm.get_sample_sets().index:
    sam = sandbox2wm.get_sample_sets().loc[s, 'samples']
    sandbox1wm.update_sample_set(sample_set_id=s, sample_ids=sam)

In [ ]:
for s in sandbox1wm.get_pair_sets().index:
    pairs = sandbox2wm.get_pair_sets().loc[s, 'pairs']
    sandbox1wm.update_pair_set(pair_set_id=s, pair_ids=pairs)

In [ ]:
from genepy import terra

samples_to_remove = {'CDS-NE61ys', 'CDS-YANwQB', 'CDS-ywnbJT', 'CDS-P4ZoH4', 'CDS-d2xe4x', 'CDS-A7rsOJ', 'CDS-CCAK2f', 'CDS-ezLdbO', 'CDS-TUYedU', 'CDS-GNOJc5', 'CDS-P5LOnq'}

terra.removeSamples(sandbox1ws, samples_to_remove)

In [ ]:
# since we are using pair and pair_set for WES from now on,
# we need to run aggreate_CN on pair_set = all
# sanger lines have been rerun as pairs, and
# the following steps move non-sanger line's seg files over to the pairs page

sandbox1ws = 'broad-firecloud-ccle/DepMap_WES_CN_hg38'
sandbox1wm = dm.WorkspaceManager(sandbox1ws).disable_hound()

wespairs = sandbox1wm.get_pairs()
wessamples = sandbox1wm.get_samples()

In [ ]:
wespairs[wespairs.called_copy_ratio_segments_tumor.isnull()].index

In [ ]:
wessamples[wessamples.called_copy_ratio_segments_tumor.isnull()].index

In [ ]:
str(wessamples.loc['CDS-pgiLpD', 'called_copy_ratio_segments_tumor'])

In [ ]:
for p in wespairs[wespairs.called_copy_ratio_segments_tumor.isnull()].index:
    i = p.split('_')[0]
    if str(wessamples.loc[i, 'called_copy_ratio_segments_tumor']) != 'nan':
        wespairs.loc[p, 'called_copy_ratio_segments_tumor'] = wessamples.loc[i, 'called_copy_ratio_segments_tumor']
    else:
        print(i)

In [ ]:
sandbox1wm.update_entity_attributes('pair', wespairs)